In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Docker

- Some files in /var/lib/docker (owned by root - not even read-only access for others!)

- https://github.com/wsargent/docker-cheat-sheet
- (official?) Docker library for Python https://github.com/docker/docker-py   



#### Important Distinction

**container** - frozen snapshot   
**image** - running instance of container

### Getting Some Basic Info

In [ ]:
!docker images

In [ ]:
!docker info

## Processes

In [ ]:
@interact_manual
def run_this():
    !docker ps -a

##  Download Container

In [ ]:
!docker pull ubuntu

### List Docker Images
Images available on local PC

In [ ]:
!docker images

### Run image (creates running container)
Note a container might be designed to start, run, then automatically stop.    
If you want it to stay active, might be necessary to give it something to do!

-it option seems common

-t              : Allocate a pseudo-tty (gives a terminal running within the container)    
-i              : Keep STDIN open even if not attached (containers tend to auto shutdown if they have nothing to do. This keeps it running)

**
```docker run <image name>
docker run <image ID>
```
**

In [ ]:
!docker run ubuntu

In [ ]:
!docker run -i ubuntu

In [ ]:
!docker run -it ubuntu

### Accessing already running container
``` docker exec -it <container name> bash ```

## Rename Image

In [ ]:
#Rename image - (doesn't stick?)
!docker rename awesome_ptolemy ubuntu1

## Making new image from container (so changes stick)
https://www.digitalocean.com/community/tutorials/how-to-install-and-use-docker-on-ubuntu-16-04

- Make changes to running container (e.g. by running apt-get inside it)
- Exit from the container ($ exit)
- Commit the changes using **docker commit** (causing new image to be created, incorporating the changes)

```docker commit <image id of updated image> <name for new container>```   
docker commit 9c09acd48a25 ubuntu-with-newthing   

e.g. (worked) -m adds optional message, -a optional author     
docker commit -m "added nano and python" -a "Alan" ad3e7f6c5504 ubuntu-alan

In [ ]:
docker commit -m "added nano and python" -a "Alan" ad3e7f6c5504 ubuntu-alan

## Transferring Images
(witout using repository/ Docker website)    
https://stackoverflow.com/questions/23935141/how-to-copy-docker-images-from-one-host-to-another-without-via-repository

- save the image using **docker save** (creates a tar file)
- load the image on other machine using **docker load**

```
docker save -o <image tar file path (inc filename)> <docker image name to be saved>

e.g. docker save -o ubuntu_image1.tar ubuntu1

docker load -i <path to image tar file>

```

### Python's own Docker module
https://github.com/docker/docker-py     

sudo pip install docker   
 or   
pip install --user docker   

#### Create Docker Client (local)

In [ ]:
import docker
client = docker.from_env()

#### Create Docker Client (remote)

According to, https://stackoverflow.com/questions/35695366/connect-to-docker-hosted-on-remote-server, something like
```
client = docker.Client(base_url='<https_url>', tls=False)

client = docker.Client(base_url='<https_url>', tls=True)
```

Not sure if right though (might be out-of-date). http://docker-py.readthedocs.io/en/stable/client.html suggests:

client = docker.DockerClient(base_url='unix://var/run/docker.sock')


#### Some info from client (that's been named 'client')

In [ ]:
client.containers.list()

In [ ]:
client.containers.list(all=True)

In [ ]:
client.images.list()

#### Start a container

In [ ]:
# stdin_open=True, same effect as docker run -i (keeps container running)
container = client.containers.run(image='ubuntu-alan:latest', detach=True, stdin_open=True, name='myubuntu')

In [ ]:
client.containers.list()

In [ ]:
container.logs()

In [ ]:
container.status

In [ ]:
client.containers.list()

In [ ]:
container.name
container.id

In [ ]:
response = container.exec_run("pwd")

In [ ]:
container.logs()

#### Stop the Container

In [ ]:
container.stop()

### Access already running container

In [ ]:
running_container =  client.containers.get("e50d653402")

In [ ]:
response = running_container.exec_run("ls")
print response